# Experiment for dataset metadata tracking

In [1]:
# notebook configuration
COLAB = False # will trigger manual installation of packages
USE_GPU = True 

In [2]:
# basic imports
from pathlib import Path
import os
import math
from datetime import datetime
import random

In [3]:
%matplotlib inline
%config Completer.use_jedi = False
os.environ['WANDB_NOTEBOOK_NAME'] = f"experiment_{datetime.now().strftime('%Y%m%d')}.ipynb"

Now, non-stdlib imports

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import log_loss, roc_auc_score
import wandb
from wandb.xgboost import wandb_callback
from wandb.lightgbm import wandb_callback
from sklearn.impute import SimpleImputer #, KNNImputer

import seaborn as sns

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
# from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.preprocessing import RobustScaler #StandardScaler #, MinMaxScaler, MaxAbsScaler, RobustScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
# from sklearn.feature_selection import SelectKBest, f_regression
from joblib import dump, load
# feature engineering tools
# from sklearn.feature_selection import mutual_info_regression
# import featuretools as ft

from BorutaShap import BorutaShap

Now, datapath setup

In [5]:
if COLAB:
    # mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # handling datapath
    datapath = Path('/content/drive/MyDrive/kaggle/tabular_playgrounds/nov2021/')
    
else:
    # if on local machine
#     datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/')  
    root = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/nov2021/')
    datapath = root/'datasets'
    # edapath = root/'EDA'
    # modelpath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/models/')
    predpath = root/'preds'
    subpath = root/'submissions'
    
    for pth in [datapath, predpath, subpath]:
        pth.mkdir(exist_ok=True)
    


In [6]:
SEED = 42

# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(seed=SEED)

## Experiment: Dataset Parameters

In [10]:
# dataset_params will initially include either trivial class instances or loaded, precomputed artifacts
dataset_params = {
    'train_source': str(datapath/'X-RobustScaled-pca_mle-umap_embedding_20211107-n_comp10-n_neighbors15-rs42.joblib'),
    'target_source': str(datapath/'y_orig.joblib'),
    'test_source': str(datapath/'X_test-RobustScaled-pca_mle-umap_embedding_20211107-n_comp10-n_neighbors15-rs42.joblib'),
    'scaler': str(RobustScaler()),
    'pca': str(load(datapath/'pca_mle-RobustScaled_orig_trainset.joblib')),
    'umap': str(load(datapath/'umap_reducer-20211107-n_comp10-n_neighbors15-rs42-pca_mle-RobustScaled_orig_trainset.joblib')),
}   

# referring back to the already-entered attributes, specify how the pipeline was sequenced
# dataset_params['preprocessing_pipeline'] = str([dataset_params['scaler'], dataset_params['pca'], dataset_params['umap']]) # ACTUALLY this is unwieldy
dataset_params['preprocessing_pipeline'] = '[scaler, pca, umap]' # more fragile, but also more readable

# now, load the datasets and generate more metadata from them
X = load(dataset_params['train_source'])
y = load(dataset_params['target_source'])
X_test = load(dataset_params['test_source'])

dataset_params['feature_count'] = X.shape[1]
dataset_params['instance_count'] = X.shape[0]
    

## Ex-Model Config

In [11]:
# meta-config for preprocessing and cross-validation, but NOT for model parameters
exmodel_config = {
#     "feature_selector": SelectKBest,
#     "k_best": 80,
#     "feature_selection_scoring": f_regression,
#     'random_state': SEED,
#     'feature_generation': ['NaN_counts', 'SummaryStats', 'NaN_OneHots'],
#     'subsample': 1,
    'cross_val_strategy': KFold, # None for holdout, or the relevant sklearn class
    'kfolds': 5, # if 1, that means just doing holdout
    'test_size': 0.2,
    **dataset_params
#     'features_created': False,
#     'feature_creator': None,
}

In [12]:
exmodel_config

{'cross_val_strategy': sklearn.model_selection._split.KFold,
 'kfolds': 5,
 'test_size': 0.2,
 'train_source': '/media/sf/easystore/kaggle_data/tabular_playgrounds/nov2021/datasets/X-RobustScaled-pca_mle-umap_embedding_20211107-n_comp10-n_neighbors15-rs42.joblib',
 'target_source': '/media/sf/easystore/kaggle_data/tabular_playgrounds/nov2021/datasets/y_orig.joblib',
 'test_source': '/media/sf/easystore/kaggle_data/tabular_playgrounds/nov2021/datasets/X_test-RobustScaled-pca_mle-umap_embedding_20211107-n_comp10-n_neighbors15-rs42.joblib',
 'scaler': 'RobustScaler()',
 'pca': "PCA(n_components='mle', random_state=42)",
 'umap': "UMAP(n_components=10, random_state=42, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True})",
 'preprocessing_pipeline': '[scaler, pca, umap]',
 'feature_count': 10,
 'instance_count': 600000}

## Data Setup

**TODO** Write some conditional logic here to automate it -- possibly as part of a sklearn.*pipeline

In [10]:
# if exmodel_config['scaler']:
#     scaler = exmodel_config['scaler']()
#     scaler.fit_transform()

In [11]:
test_source = datapath/'test.feather'
# # test_source = altdatapath/'test-WITH-KMeans_12cluster_kmeans++_maxiter1000_rs42.feather' # altdatapath/'X_test_boruta_shap_200trials.feather'
exmodel_config['test_source'] = str(test_source)
X_test = pd.read_feather(path=test_source)
# # boruta = load(altdatapath/'boruta_shap.joblib')
# # X_test_enc = 
# # X_test = X_test.iloc[:, 1:] # only if loading the original test

In [12]:
# X_test = X_test.drop(['id'], axis=1)

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [13]:
train_source = datapath/'X_orig.feather'
# train_source = altdatapath/'train-WITH-KMeans_12cluster_kmeans++_maxiter1000_rs42.feather' #'X_boruta_shap_200trials.feather'
# exmodel_config['train_source'] = str(train_source)
X = pd.read_feather(path=train_source)
# if 'target' in X.columns:
#     X = X.drop(['target'], axis=1)
# df.index.name = 'id'
# y = np.array(df.target)
y = load(datapath/'y_orig.joblib')
# features = [x for x in df.columns if x != 'target']
# X = df[features] # passing X as a pd.DataFrame to the trainer below, rather than as an np.ndarray
# X_train = df[features]
# X.index.name = 'id'
# y.index.name = 'id'


In [14]:
scaler = RobustScaler()
X_np = scaler.fit_transform(X)

In [15]:
X = pd.DataFrame(X_np, columns=X.columns, index=X.index)

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
